In [85]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [86]:
import sys
import os
import torch
from functools import partial
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

import torch.nn as nn
from embedding import CustomEmbedding
from transformers import EncoderDecoderTransformer
from utils import padding_collate_fn

from generate_data import RandomIntegerDataset

In [87]:
n_real_tokens = 10
PAD_TOKEN_IDX = n_real_tokens
SOS_TOKEN_IDX = n_real_tokens + 1
EOS_TOKEN_IDX = n_real_tokens + 2
vocab_size = n_real_tokens + 3
D_MODEL = 16

embeddings = CustomEmbedding(vocab_size = vocab_size + 3, d_model = D_MODEL) # 3 = PAD, SOS, EOS

indices = torch.tensor([1,9])

# print(embeddings.embeddings.weight)
print(embeddings.embeddings(indices))

tensor([[-0.4959,  1.1532, -1.5517, -0.0264, -1.2842, -0.2948,  0.1843,  2.5940,
          0.2433,  0.3829,  1.2278, -0.0028, -0.4759, -0.9599,  1.4245,  0.8282],
        [-1.5257, -1.3818, -0.7827, -0.7115, -0.7646, -0.9675, -0.9097, -0.6993,
          2.6623, -0.0931,  1.1417, -0.2799, -1.7919, -0.7490,  1.1308,  0.1573]],
       grad_fn=<EmbeddingBackward0>)


In [88]:
MAX_CONTEXT_WINDOW = 50

BATCH_SIZE = 32
MIN_SEQ_LEN = 2
MAX_SEQ_LEN = min(10, MAX_CONTEXT_WINDOW)

rand_ds = RandomIntegerDataset(2, 10, 10000, [i for i in range(10)])
dataloader = DataLoader(rand_ds, batch_size = BATCH_SIZE, shuffle = True, collate_fn = partial(padding_collate_fn, pad_token_idx = PAD_TOKEN_IDX))

In [89]:
input, label = next(iter(dataloader))
print(input[0])
print(input[1])
print(label)

tensor([[ 8,  5,  0,  2,  6, 10, 10, 10, 10, 10],
        [ 6,  6,  1,  7, 10, 10, 10, 10, 10, 10],
        [ 1,  5,  1,  1, 10, 10, 10, 10, 10, 10],
        [ 4,  4,  8,  3, 10, 10, 10, 10, 10, 10],
        [ 2,  1,  8,  2,  4,  0,  0,  3,  1, 10],
        [ 5,  7,  6,  0,  0, 10, 10, 10, 10, 10],
        [ 6,  8,  0,  1,  6, 10, 10, 10, 10, 10],
        [ 5,  4,  4,  3,  3,  1,  6, 10, 10, 10],
        [ 0,  2,  8,  0,  4,  3,  9,  1,  1, 10],
        [ 9,  2,  7,  5,  9,  0, 10, 10, 10, 10],
        [ 2,  5,  3,  4, 10, 10, 10, 10, 10, 10],
        [ 9,  9,  7,  5,  0,  4, 10, 10, 10, 10],
        [ 7,  2,  3,  7,  2,  9,  9,  4,  7, 10],
        [ 8,  8, 10, 10, 10, 10, 10, 10, 10, 10],
        [ 5,  0,  9,  5,  4,  6,  0,  5,  0, 10],
        [ 8,  7,  0,  7, 10, 10, 10, 10, 10, 10],
        [ 0,  3,  1,  0,  9,  4,  4,  6,  7,  0],
        [ 0,  9,  3, 10, 10, 10, 10, 10, 10, 10],
        [ 2,  2,  1,  7,  2,  1,  0,  0,  1, 10],
        [ 2,  7,  7,  0,  2, 10, 10, 10, 10, 10],


In [90]:
loss_fn = nn.CrossEntropyLoss(ignore_index = PAD_TOKEN_IDX, reduction = 'sum')

model = EncoderDecoderTransformer(
                    embeddings = embeddings, 
                    vocab_size = vocab_size, 
                    d_model = D_MODEL, 
                    num_attention_heads = 4, 
                    num_encoder_layers = 1, 
                    num_decoder_layers = 1, 
                    dim_feedforward = 32, 
                    max_context_window = MAX_CONTEXT_WINDOW)

optim = torch.optim.SGD(params = model.parameters(), lr = 1e-3, momentum = 0.9, weight_decay = 1e-4)

In [91]:
def run_train_epoch(dataloader: DataLoader, model: nn.Module, loss_fn: nn.Module, optimizer: torch.optim.Optimizer):
    
    model.train()

    epoch_loss = 0.0

    for i, ((source, target), label) in tqdm(enumerate(dataloader)):

        # FORWARD
        pred_logits = model(source, target)
        # CrossEntropyLoss (loss_fn) only takes 2D predictions (n_batch * seq_len, vocab_size) and 1D labels (n_batch * seq_len)
        batch_loss = loss_fn(pred_logits.view(-1, vocab_size), label.view(-1))

        # LOG
        epoch_loss += batch_loss.item()

        # BACKWARD
        batch_loss.backward()

        # OPTIMIZE
        optimizer.step()
        optimizer.zero_grad()

    average_epoch_loss = epoch_loss / len(dataloader.dataset)

    return average_epoch_loss

def run_validation_loop(dataloader: DataLoader, model: nn.Module, loss_fn: nn.Module):
    
    model.eval()

In [ ]:
EPOCHS = 10

training_losses = list()
validation_losses = list()

for i in range(EPOCHS):
    print(f'Running epoch {i+1}...')

    training_loss = run_train_epoch(dataloader, model, loss_fn, optim)
    training_losses.append(training_loss)

    # validation_loss = run_validation_loop(dataloader, model, loss_fn)
    # validation_losses.append(validation_loss)

print(training_losses)

Running epoch 1...


313it [00:02, 130.59it/s]


Running epoch 2...


313it [00:02, 132.68it/s]


Running epoch 3...


...

Running epoch 9...


313it [00:02, 141.00it/s]


Running epoch 10...


313it [00:02, 139.14it/s]

[4.998334381866455, 3.1808130752563475, 2.8981128993988037, 2.335165265083313, 2.150253467941284, 2.341424671936035, 2.176064828491211, 2.0349822174072267, 1.8267414445877075, 2.13696817111969]
